# Using RAGAS to evaluate your RAG system
- **RAGAS** is a strong Python module used to evaluate the RAG systems. One of the most popular techniques in RAG evaluation is **model-based evaluation**. This method uses a powerful/commercial LLM model to measure the similarity between the RAG’s generated responses and the expected results.
- I use **Google VertexAI** and their `gemini-1.5-flash` LLM model to evaluate my RAG system in this lab.
- The main components in this system include:
  - LLM model: `DeepSeek R1 Distill LLama3 70B`
  - Embedding model: `sentence-transformers/paraphrase-multilingual-mpnet-base-v2`
  - Evaluation LLM model: `gemini-1.5-flash`

In [1]:
from ragas.llms import llm_factory
%load_ext autoreload
%autoreload 2

/opt/anaconda3/envs/llm-evaluation/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import sys
import os

In [3]:
sys.path

['/opt/anaconda3/envs/llm-evaluation/lib/python310.zip',
 '/opt/anaconda3/envs/llm-evaluation/lib/python3.10',
 '/opt/anaconda3/envs/llm-evaluation/lib/python3.10/lib-dynload',
 '',
 '/opt/anaconda3/envs/llm-evaluation/lib/python3.10/site-packages']

## Prepare the documents

In [4]:
from langchain_community.document_loaders import PyPDFLoader
from collections import defaultdict
from langchain.schema import Document

In [5]:
directory_path = "./../data/vks/pdf/vi"  # Change this to your directory
md_files = []

for root, _, files in os.walk(directory_path):
    for file in files:
        if file.endswith(".pdf"):
            md_files.append(os.path.join(root, file))

print(md_files)

['./../data/vks/pdf/vi/03-mo-hinh-hoat-dong.pdf', './../data/vks/pdf/vi/02-nlb.pdf', './../data/vks/pdf/vi/04-thong-bao-cap-nhat-release-note.pdf', './../data/vks/pdf/vi/01-vks-la-gi.pdf']


In [6]:
docs = []

for file in md_files:
    loader = PyPDFLoader(file)

    data = loader.load()
    docs.extend(data)

docs

[Document(metadata={'producer': 'Skia/PDF m133', 'creator': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/133.0.0.0 Safari/537.36', 'creationdate': '2025-02-18T07:22:18+00:00', 'title': '02-mo-hinh-hoat-dong', 'moddate': '2025-02-18T07:22:18+00:00', 'source': './../data/vks/pdf/vi/03-mo-hinh-hoat-dong.pdf', 'total_pages': 2, 'page': 0, 'page_label': '1'}, page_content='3. Mô hình hoạt động của VKS\nVNGCloud Kubernetes Service (VKS) hỗ trợ 2 loại cluster chính là Public Cluster và Private\nCluster. Đối với node group, VKS hỗ trợ 2 loại node group chính là Public Node Group và\nPrivate Node Group. Từ đó, khách hàng có thể kết hợp giữa các loại cluster và node group để\ntạo ra các mô hình hoạt động phù hợp với yêu cầu của ứng dụng, cụ thể:\nPublic Cluster với Public Node Group: Các node trong nodegroup có địa chỉ IP Public\nriêng, kết đến nối ControlPlane thông qua IP Public của Kubernetes API Server. Điều này\ndẫn rủi ro bảo mật cao hơn. T

In [7]:
# Group documents by source
merged_docs = defaultdict(str)
for doc in docs:
    merged_docs[doc.metadata["source"]] += (
        doc.page_content + "\n"
    )  # Concatenating with newline

In [8]:
new_docs = []

for source, doc in merged_docs.items():
    new_doc = Document(page_content=doc, metadata={"source": source})

    new_docs.append(new_doc)

In [9]:
new_docs

[Document(metadata={'source': './../data/vks/pdf/vi/03-mo-hinh-hoat-dong.pdf'}, page_content='3. Mô hình hoạt động của VKS\nVNGCloud Kubernetes Service (VKS) hỗ trợ 2 loại cluster chính là Public Cluster và Private\nCluster. Đối với node group, VKS hỗ trợ 2 loại node group chính là Public Node Group và\nPrivate Node Group. Từ đó, khách hàng có thể kết hợp giữa các loại cluster và node group để\ntạo ra các mô hình hoạt động phù hợp với yêu cầu của ứng dụng, cụ thể:\nPublic Cluster với Public Node Group: Các node trong nodegroup có địa chỉ IP Public\nriêng, kết đến nối ControlPlane thông qua IP Public của Kubernetes API Server. Điều này\ndẫn rủi ro bảo mật cao hơn. Tuy nhiên bạn hoàn toàn có thể hạn chế truy cập đến\nControlPlane thông qua tính năng Whitelist IP. Một điều lưu ý là bạn cần đảm bảo không\nhạn chế truy cập từ các IP của các node trong nodegroup.\nPublic Cluster với Private Node Group: Các node trong nodegroup không có địa chỉ IP\nPublic riêng, không thể kết nối trực tiếp đế

In [10]:
len(new_docs)

4

## Chunking and save into vector-store database

In [11]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.vectorstores import InMemoryVectorStore

In [12]:
embeddings_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2",
)
vector_store = InMemoryVectorStore(embeddings_model)

In [13]:
chunk_size = 7000
chunk_overlap = 6800
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size, chunk_overlap=chunk_overlap
)

In [14]:
splits = text_splitter.split_documents(new_docs)

In [15]:
vector_store.add_documents(splits)  # save documents into vectorstore

['a6cfbf78-2b49-442b-9247-dcb76dd9bec2',
 '74e7c966-9b8d-49fd-ac47-fb2d56adcfa4',
 '3fe68ed2-952b-416c-8543-136f1cb82577',
 '7bb9aa8f-e577-473e-848a-1ede8ea700fc',
 'a9524ea3-5a01-4d43-949f-ef29c02487c3',
 'eb62adb9-f96c-41ba-b84d-167dbf1b6af3',
 '1282be68-c9b7-4fc7-9330-3c2a45cd6be4',
 '00a57724-4e87-4cc5-ab44-8b8ae1568684',
 '3c7c7257-2915-47da-ab75-36c0f6a704e6',
 '3a07a394-f2f1-4025-a8fb-c8099891ef71',
 'fa5f8f6c-0edd-444a-a975-a9be51da3b29',
 '0679892e-f59f-4c29-8b16-607d1f77db8a',
 '47ff656e-46ac-4c54-a7af-40be3d22573d',
 '13fb00fa-7652-4999-996a-4d387b3543ea',
 '81c9893e-e039-4a5b-af19-0c6be3becf4d',
 '296889f4-49c6-4289-b6ce-6e669cea8f70',
 'b891398f-175a-4a40-8a0a-83518961b0a4',
 'd1cce114-3eeb-4f0b-9763-bd65e0517696',
 '78eda6a6-31c3-4ec7-adcc-787eb50c4127',
 '060fbe22-e25c-4490-8729-f41a485f5c6d',
 '8c8c950c-1bb6-43e7-8b64-5f8089ebd0ca',
 '9606950e-3891-48da-9e12-a15dd0566c85',
 '75d64f55-762c-4749-b02f-669e25199dbe',
 '54092ddf-e034-4436-a32d-731eaa6cf9b2',
 '02d4cb70-9f39-

In [16]:
retriever = vector_store.as_retriever()

In [17]:
query = "VKS là gì?"
for i, doc in enumerate(retriever.invoke(query)):
    print(f"Document {i}:\n{doc}", end="\n\n")

Document 0:
page_content='1. Giới thiệu về VNGCloud Kubernetes
Service - VKS
1.1. VNGCloud Kubernetes Service (VKS) là gì?
VNGCloud Kubernetes Service (hay còn gọi là VKS) là một dịch vụ quản lý Kubernetes được cung
cấp bởi VNGCloud. VKS giúp bạn triển khai và quản lý các ứng dụng dựa trên container một cách
dễ dàng và hiệu quả. VKS giúp bạn tập trung vào việc phát triển ứng dụng mà không cần quan tâm
đến việc quản lý Control Plane của Kubernetes.
VKS (VNGCloud Kubernetes Service) là một dịch vụ được quản lý trên VNGCloud giúp bạn đơn
giản hóa quá trình triển khai và quản lý các ứng dụng dựa trên container. Kubernetes là một nền
tảng mã nguồn mở được phát triển bởi Google, được sử dụng rộng rãi để quản lý và triển khai các
ứng dụng container trên môi trường phân tán.
1.2. Những điểm nổi bật của VKS
Những điểm nổi bật của dịch vụ VKS có thể kể đến gồm:
Quản lý Control Plane hoàn toàn tự động (Fully Managed control plane): VKS sẽ giải phóng
bạn khỏi gánh nặng quản lý Control Plane của Ku

## Configure some environment variables

In [18]:
from modules import utils

In [19]:
ENV_FILE_PATH = (
    "/Users/cuongdm8499/Me/git-cuongpiger/secret/work/vngcloud/ai-platform/env"
)
MODEL_NAME = "deepseek-ai/DeepSeek-R1-Distill-Llama-70B"
MAX_TOKENS = 32700
COLLECTION_NAME = "my_docs"


os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = (
    # "/home/stackops/secret/work/vngcloud/ai-platform/vertex-ai-credential.json"
    "/Users/cuongdm8499/Me/git-cuongpiger/secret/work/vngcloud/ai-platform/vertex-ai-credential.json"
)

envs = utils.load_env_to_dict(ENV_FILE_PATH)

# Configure PostgreSQL vectorstore

In [20]:
from langchain_postgres.vectorstores import PGVector

In [21]:
postgre_vector_store = PGVector(
    embeddings=embeddings_model,
    collection_name=COLLECTION_NAME,
    connection=envs["POSTGRESQL_URI_2"],
    use_jsonb=True,
)

## Setup RAG chain

In [22]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from llm import gemini, deepseek

## Define Genimi factory wrapper

In [23]:
llm_gemini = gemini.Gemini()
llm_deepseek = deepseek.DeepSeek(
    envs["VLLM_HOST_URL_2"] + "/v1/",
    MODEL_NAME,
    MAX_TOKENS,
)

/opt/anaconda3/envs/llm-evaluation/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3579: UserWarning: Parameters {'stop'} should be specified explicitly. Instead they were passed in as part of `model_kwargs` parameter.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [24]:
# Define prompt template
template = """You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Use two sentences maximum and keep the answer concise.
Question: {question}
Context: {context}
Answer:
"""
prompt = ChatPromptTemplate.from_template(template)
gemini_rag = llm_gemini.rag_chain(prompt, retriever)
deepseek_rag = llm_deepseek.rag_chain(prompt, retriever)

## Preparing the Evaluation Data

In [25]:
from evaluation import evaluation

In [26]:
questions = [
    "VKS có những tính năng gì nổi bật?",
    "VKS là gì?",
    "So sánh public và private cluster trong VKS",
]

ground_truths = [
    """
    Dịch vụ VKS (VNGCloud Kubernetes Service) có những tính năng nổi bật sau:

    Quản lý Control Plane hoàn toàn tự động (Fully Managed control plane): VKS giúp bạn giải phóng khỏi gánh nặng quản lý Control Plane của Kubernetes, giúp bạn tập trung vào việc phát triển ứng dụng.
    Hỗ trợ các phiên bản Kubernetes mới nhất: VKS luôn cập nhật những phiên bản Kubernetes mới nhất để đảm bảo bạn luôn tận dụng được những tính năng tiên tiến nhất.
    Kubernetes Networking: VKS tích hợp Calico CNI, mang lại tính hiệu quả và bảo mật cao.
    Upgrade seamlessly: VKS hỗ trợ nâng cấp giữa các phiên bản Kubernetes một cách dễ dàng và nhanh chóng.
    Scaling & Healing Automatically: VKS tự động mở rộng Node group khi cần thiết và tự động sửa lỗi khi node gặp vấn đề, giúp bạn tiết kiệm thời gian và công sức quản lý.
    Giảm chi phí và nâng cao độ tin cậy: VKS triển khai Control Plane của Kubernetes ở chế độ sẵn sàng cao và hoàn toàn miễn phí.
    Tích hợp Blockstore Native (Container Storage Interface - CSI): VKS cho phép bạn quản lý Blockstore thông qua YAML của Kubernetes, cung cấp lưu trữ bền vững cho container.
    Tích hợp Load Balancer (Network Load Balancer, Application Load Balancer): VKS cung cấp khả năng quản lý NLB/ALB thông qua YAML của Kubernetes, giúp bạn dễ dàng expose Service trong Kubernetes ra bên ngoài.
    Nâng cao bảo mật: VKS cho phép bạn tạo Private Node Group với chỉ Private IP và kiểm soát quyền truy cập vào cluster.
    Ngoài ra, VKS còn có các ưu điểm sau:

    Dễ sử dụng: VKS cung cấp giao diện đơn giản và dễ sử dụng.
    Chi phí hợp lý: VKS cung cấp mức giá cạnh tranh cho các dịch vụ của mình.
    Bạn có muốn tìm hiểu thêm về tính năng nào cụ thể của VKS không?
    """,
    """
    VKS là viết tắt của VNGCloud Kubernetes Service.

    Nó là một dịch vụ quản lý Kubernetes được cung cấp bởi VNGCloud.

    VKS giúp bạn triển khai và quản lý các ứng dụng dựa trên container một cách dễ dàng và hiệu quả. Nó giải phóng bạn khỏi gánh nặng quản lý Control Plane của Kubernetes, giúp bạn tập trung vào việc phát triển ứng dụng.

    Nói cách khác, VKS là một giải pháp giúp bạn dễ dàng sử dụng Kubernetes mà không cần phải tự cài đặt và quản lý nó.
    """,
    """
    Public Cluster và Private Cluster là hai loại cụm Kubernetes chính được hỗ trợ bởi VKS. Mỗi loại có ưu nhược điểm riêng, phù hợp với những nhu cầu bảo mật và sử dụng khác nhau.

    Dưới đây là bảng so sánh chi tiết giữa Public Cluster và Private Cluster trong VKS:

    Tính năng	Public Cluster	Private Cluster
    Kết nối	Sử dụng địa chỉ Public IP để giao tiếp giữa nodes, control plane, client và các dịch vụ khác trong VNG Cloud	Sử dụng địa chỉ Private IP để giao tiếp giữa nodes, control plane, client và các dịch vụ khác trong VNG Cloud
    Bảo mật	Bảo mật trung bình do các kết nối sử dụng Public IP	Bảo mật cao hơn với tất cả kết nối đều private và giới hạn truy cập
    Quản lý truy cập	Khó kiểm soát hơn, có thể quản lý truy cập thông qua tính năng Whitelist	Kiểm soát truy cập chặt chẽ, mọi kết nối đều nằm trong mạng private của VNG Cloud
    Khả năng mở rộng (AutoScaling)	Dễ dàng mở rộng thông qua tính năng Auto Scaling	Dễ dàng mở rộng thông qua tính năng Auto Scaling
    Khả năng tự hồi phục (AutoHealing)	Tự động phát hiện lỗi và khởi động lại node (Auto Healing)	Tự động phát hiện lỗi và khởi động lại node (Auto Healing)
    Khả năng truy cập từ bên ngoài	Dễ dàng truy cập từ bất kỳ đâu	Truy cập từ bên ngoài phải qua các giải pháp bảo mật khác
    Cấu hình và triển khai	Đơn giản hơn do không yêu cầu thiết lập mạng nội bộ	Phức tạp hơn, yêu cầu cấu hình mạng private và bảo mật
    Chi phí	Thường thấp hơn do không cần thiết lập cơ sở hạ tầng bảo mật phức tạp	Chi phí cao hơn do yêu cầu thêm các thành phần bảo mật và quản lý
    Tính linh hoạt	Cao, dễ dàng thay đổi và truy cập các dịch vụ	Linh hoạt hơn trong các ứng dụng yêu cầu bảo mật, nhưng ít linh hoạt hơn cho các ứng dụng yêu cầu truy cập từ bên ngoài
    Phù hợp với	Các ứng dụng không yêu cầu bảo mật cao và cần sự linh hoạt, truy cập từ nhiều địa điểm	Các ứng dụng yêu cầu bảo mật cao, tuân thủ nghiêm ngặt các quy định về bảo mật và quyền riêng tư
    Kết luận:

    Public Cluster phù hợp cho các ứng dụng không yêu cầu bảo mật cao và cần sự linh hoạt, truy cập từ nhiều địa điểm. Dễ dàng triển khai và quản lý nhưng có rủi ro bảo mật cao hơn.
    Private Cluster phù hợp cho các ứng dụng yêu cầu bảo mật cao, tuân thủ nghiêm ngặt các quy định về bảo mật và quyền riêng tư. Mang lại kết nối ổn định và bảo mật, nhưng yêu cầu cấu hình và quản lý phức tạp hơn, cũng như chi phí cao hơn.
    Bạn cần cân nhắc kỹ lưỡng nhu cầu của mình để lựa chọn loại cluster phù hợp nhất.
    """,
]

gemini_dataset = evaluation.prepare_evaluation_dataset(
    questions, ground_truths, gemini_rag, retriever
)

deepseek_dataset = evaluation.prepare_evaluation_dataset(
    questions, ground_truths, deepseek_rag, retriever
)

# Prepare the evaluator model using VertexAI

In [27]:
from langchain_google_vertexai import ChatVertexAI

In [28]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = (
    # "/home/stackops/secret/work/vngcloud/ai-platform/vertex-ai-credential.json"
    "/Users/cuongdm8499/Me/git-cuongpiger/secret/work/vngcloud/ai-platform/vertex-ai-credential.json"
)

In [29]:
evaluator_llm = ChatVertexAI(model_name="gemini-1.5-flash", temperature=0)

## Evaluating the RAG application


In [30]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
)

In [31]:
gemini_result = evaluate(
    llm=evaluator_llm,
    embeddings=embeddings_model,
    dataset=gemini_dataset,
    metrics=[
        context_precision,
        context_recall,
        faithfulness,
        answer_relevancy,
    ],
)

deepseek_result = evaluate(
    llm=evaluator_llm,
    embeddings=embeddings_model,
    dataset=deepseek_dataset,
    metrics=[
        context_precision,
        context_recall,
        faithfulness,
        answer_relevancy,
    ],
)

print("Gemini result:", gemini_result)
print("Deepseek result:", deepseek_result)

Evaluating: 100%|██████████| 12/12 [00:05<00:00,  2.31it/s]


Gemini result: {'context_precision': 1.0000, 'context_recall': 0.7786, 'faithfulness': 1.0000, 'answer_relevancy': 0.8655}
Deepseek result: {'context_precision': 1.0000, 'context_recall': 0.7786, 'faithfulness': 1.0000, 'answer_relevancy': 0.9252}


In [32]:
gemini_result_df = gemini_result.to_pandas()
deepseek_result_df = deepseek_result.to_pandas()

gemini_result_df["llm_model"] = "gemini-1.5-flash"
deepseek_result_df["llm_model"] = "deepseek-r1-distill-llama-70b"

In [33]:
gemini_result_df

,user_input,retrieved_contexts,response,reference,context_precision,context_recall,faithfulness,answer_relevancy,llm_model
0,VKS có những tính năng gì nổi bật?,[1. Giới thiệu về VNGCloud Kubernetes\nService...,VKS cung cấp nhiều tính năng nổi bật như quản ...,\n Dịch vụ VKS (VNGCloud Kubernetes Service...,1.0,0.785714,1.0,0.917253,gemini-1.5-flash
1,VKS là gì?,[1. Giới thiệu về VNGCloud Kubernetes\nService...,VKS là viết tắt của VNGCloud Kubernetes Servic...,\n VKS là viết tắt của VNGCloud Kubernetes ...,1.0,0.800000,1.0,0.945761,gemini-1.5-flash
2,So sánh public và private cluster trong VKS,[3. Mô hình hoạt động của VKS\nVNGCloud Kubern...,Public Cluster sử dụng địa chỉ IP Public để gi...,\n Public Cluster và Private Cluster là hai...,1.0,0.750000,1.0,0.733616,gemini-1.5-flash


In [34]:
deepseek_result_df

,user_input,retrieved_contexts,response,reference,context_precision,context_recall,faithfulness,answer_relevancy,llm_model
0,VKS có những tính năng gì nổi bật?,[1. Giới thiệu về VNGCloud Kubernetes\nService...,VKS có các tính năng nổi bật như quản lý Contr...,\n Dịch vụ VKS (VNGCloud Kubernetes Service...,1.0,0.785714,1.0,0.928116,deepseek-r1-distill-llama-70b
1,VKS là gì?,[1. Giới thiệu về VNGCloud Kubernetes\nService...,VKS là dịch vụ quản lý Kubernetes của VNGCloud...,\n VKS là viết tắt của VNGCloud Kubernetes ...,1.0,0.800000,1.0,0.902287,deepseek-r1-distill-llama-70b
2,So sánh public và private cluster trong VKS,[3. Mô hình hoạt động của VKS\nVNGCloud Kubern...,Public cluster trong VKS sử dụng địa chỉ IP cô...,\n Public Cluster và Private Cluster là hai...,1.0,0.750000,1.0,0.945119,deepseek-r1-distill-llama-70b


# Evaluate knowledge base from PostgreSQL

In [35]:
# Prepare postgreSQL vector store retriever
retriever_postgresql = postgre_vector_store.as_retriever()

In [36]:
gemini_rag_postgre = llm_gemini.rag_chain(prompt, retriever_postgresql)
deepseek_rag_postgre = llm_deepseek.rag_chain(prompt, retriever_postgresql)

gemini_dataset_postgre = evaluation.prepare_evaluation_dataset(
    questions, ground_truths, gemini_rag_postgre, retriever_postgresql
)

deepseek_dataset_postgre = evaluation.prepare_evaluation_dataset(
    questions, ground_truths, deepseek_rag_postgre, retriever_postgresql
)

In [37]:
gemini_result_postgreq = evaluate(
    llm=evaluator_llm,
    embeddings=embeddings_model,
    dataset=gemini_dataset_postgre,
    metrics=[
        context_precision,
        context_recall,
        faithfulness,
        answer_relevancy,
    ],
)

deepseek_result_postgreq = evaluate(
    llm=evaluator_llm,
    embeddings=embeddings_model,
    dataset=deepseek_dataset_postgre,
    metrics=[
        context_precision,
        context_recall,
        faithfulness,
        answer_relevancy,
    ],
)

print("Gemini result:", gemini_result_postgreq)
print("Deepseek result:", deepseek_result_postgreq)

Evaluating: 100%|██████████| 12/12 [00:06<00:00,  1.92it/s]


Gemini result: {'context_precision': 0.9167, 'context_recall': 0.8786, 'faithfulness': 1.0000, 'answer_relevancy': 0.8807}
Deepseek result: {'context_precision': 0.9167, 'context_recall': 0.8786, 'faithfulness': 1.0000, 'answer_relevancy': 0.8142}


In [38]:
gemini_result_df_postgre = gemini_result_postgreq.to_pandas()
deepseek_result_df_postgre = deepseek_result_postgreq.to_pandas()

gemini_result_df_postgre["llm_model"] = "gemini-1.5-flash"
deepseek_result_df_postgre["llm_model"] = "deepseek-r1-distill-llama-70b"

In [39]:
gemini_result_df_postgre

,user_input,retrieved_contexts,response,reference,context_precision,context_recall,faithfulness,answer_relevancy,llm_model
0,VKS có những tính năng gì nổi bật?,[VKS là gì?\n\nGiới thiệu VKS\n\nVKS (VNGCloud...,VKS cung cấp nhiều tính năng nổi bật như quản ...,\n Dịch vụ VKS (VNGCloud Kubernetes Service...,1.00,0.785714,1.0,0.917253,gemini-1.5-flash
1,VKS là gì?,[VKS là gì?\n\nGiới thiệu VKS\n\nVKS (VNGCloud...,VKS là viết tắt của VNGCloud Kubernetes Servic...,\n VKS là viết tắt của VNGCloud Kubernetes ...,0.75,1.000000,1.0,0.930351,gemini-1.5-flash
2,So sánh public và private cluster trong VKS,[Public Cluster và Private Cluster\n\n1. Publi...,Public Cluster cho phép các node kết nối trực ...,\n Public Cluster và Private Cluster là hai...,1.00,0.850000,1.0,0.794354,gemini-1.5-flash


In [40]:
deepseek_result_df_postgre

,user_input,retrieved_contexts,response,reference,context_precision,context_recall,faithfulness,answer_relevancy,llm_model
0,VKS có những tính năng gì nổi bật?,[VKS là gì?\n\nGiới thiệu VKS\n\nVKS (VNGCloud...,VKS (VNGCloud Kubernetes Service) có các tính ...,\n Dịch vụ VKS (VNGCloud Kubernetes Service...,1.00,0.785714,1.0,0.786562,deepseek-r1-distill-llama-70b
1,VKS là gì?,[VKS là gì?\n\nGiới thiệu VKS\n\nVKS (VNGCloud...,VKS (VNGCloud Kubernetes Service) là dịch vụ q...,\n VKS là viết tắt của VNGCloud Kubernetes ...,0.75,1.000000,1.0,0.780747,deepseek-r1-distill-llama-70b
2,So sánh public và private cluster trong VKS,[Public Cluster và Private Cluster\n\n1. Publi...,Public Cluster và Private Cluster trong VKS kh...,\n Public Cluster và Private Cluster là hai...,1.00,0.850000,1.0,0.875441,deepseek-r1-distill-llama-70b


## Concat these dataframes together

In [41]:
import pandas as pd

In [42]:
gemini_result_df_postgre["knowledge_source"] = "PostgreSQL"
deepseek_result_df_postgre["knowledge_source"] = "PostgreSQL"

In [43]:
gemini_result_df["knowledge_source"] = "InMemory"
deepseek_result_df["knowledge_source"] = "InMemory"

In [44]:
df = pd.concat(
    [
        gemini_result_df,
        deepseek_result_df,
        gemini_result_df_postgre,
        deepseek_result_df_postgre,
    ]
).reset_index(drop=True)

In [45]:
df

,user_input,retrieved_contexts,response,reference,context_precision,context_recall,faithfulness,answer_relevancy,llm_model,knowledge_source
0,VKS có những tính năng gì nổi bật?,[1. Giới thiệu về VNGCloud Kubernetes\nService...,VKS cung cấp nhiều tính năng nổi bật như quản ...,\n Dịch vụ VKS (VNGCloud Kubernetes Service...,1.00,0.785714,1.0,0.917253,gemini-1.5-flash,InMemory
1,VKS là gì?,[1. Giới thiệu về VNGCloud Kubernetes\nService...,VKS là viết tắt của VNGCloud Kubernetes Servic...,\n VKS là viết tắt của VNGCloud Kubernetes ...,1.00,0.800000,1.0,0.945761,gemini-1.5-flash,InMemory
2,So sánh public và private cluster trong VKS,[3. Mô hình hoạt động của VKS\nVNGCloud Kubern...,Public Cluster sử dụng địa chỉ IP Public để gi...,\n Public Cluster và Private Cluster là hai...,1.00,0.750000,1.0,0.733616,gemini-1.5-flash,InMemory
3,VKS có những tính năng gì nổi bật?,[1. Giới thiệu về VNGCloud Kubernetes\nService...,VKS có các tính năng nổi bật như quản lý Contr...,\n Dịch vụ VKS (VNGCloud Kubernetes Service...,1.00,0.785714,1.0,0.928116,deepseek-r1-distill-llama-70b,InMemory
4,VKS là gì?,[1. Giới thiệu về VNGCloud Kubernetes\nService...,VKS là dịch vụ quản lý Kubernetes của VNGCloud...,\n VKS là viết tắt của VNGCloud Kubernetes ...,1.00,0.800000,1.0,0.902287,deepseek-r1-distill-llama-70b,InMemory
5,So sánh public và private cluster trong VKS,[3. Mô hình hoạt động của VKS\nVNGCloud Kubern...,Public cluster trong VKS sử dụng địa chỉ IP cô...,\n Public Cluster và Private Cluster là hai...,1.00,0.750000,1.0,0.945119,deepseek-r1-distill-llama-70b,InMemory
6,VKS có những tính năng gì nổi bật?,[VKS là gì?\n\nGiới thiệu VKS\n\nVKS (VNGCloud...,VKS cung cấp nhiều tính năng nổi bật như quản ...,\n Dịch vụ VKS (VNGCloud Kubernetes Service...,1.00,0.785714,1.0,0.917253,gemini-1.5-flash,PostgreSQL
7,VKS là gì?,[VKS là gì?\n\nGiới thiệu VKS\n\nVKS (VNGCloud...,VKS là viết tắt của VNGCloud Kubernetes Servic...,\n VKS là viết tắt của VNGCloud Kubernetes ...,0.75,1.000000,1.0,0.930351,gemini-1.5-flash,PostgreSQL
8,So sánh public và private cluster trong VKS,[Public Cluster và Private Cluster\n\n1. Publi...,Public Cluster cho phép các node kết nối trực ...,\n Public Cluster và Private Cluster là hai...,1.00,0.850000,1.0,0.794354,gemini-1.5-flash,PostgreSQL
9,VKS có những tính năng gì nổi bật?,[VKS là gì?\n\nGiới thiệu VKS\n\nVKS (VNGCloud...,VKS (VNGCloud Kubernetes Service) có các tính ...,\n Dịch vụ VKS (VNGCloud Kubernetes Service...,1.00,0.785714,1.0,0.786562,deepseek-r1-distill-llama-70b,PostgreSQL


In [46]:
def highlight_cells(value):
    """Color cells based on score value."""
    if value < 0.5:
        return "background-color: red; color: white;"  # Low Score (Bad)
    elif 0.5 <= value < 0.8:
        return "background-color: orange; color: black;"  # Medium Score (Warning)
    else:
        return "background-color: lightgreen; color: black;"  # High Score (Good)


def truncate_text(value):
    """Truncate text values longer than 200 characters."""

    if isinstance(value, str) and len(value) > 200:
        return value[:200]

    if isinstance(value, list):
        return "\n\n\n\n".join(v[:100] for v in value)

    return value


# Apply styling
styled_df = df.map(truncate_text)  # Truncate text fields
styled_df = styled_df.style.map(
    highlight_cells,
    subset=["context_precision", "context_recall", "faithfulness", "answer_relevancy"],
)

# Display in Jupyter Notebook
styled_df

,user_input,retrieved_contexts,response,reference,context_precision,context_recall,faithfulness,answer_relevancy,llm_model,knowledge_source
0,VKS có những tính năng gì nổi bật?,"1. Giới thiệu về VNGCloud Kubernetes Service - VKS 1.1. VNGCloud Kubernetes Service (VKS) là gì? VNG 3. Mô hình hoạt động của VKS VNGCloud Kubernetes Service (VKS) hỗ trợ 2 loại cluster chính là Public cho các tài nguyên Service có thuộc tính serviceType là LoadBalancer . Mặc định, một cụm Kubern VNGCloud Portal. vks.vngcloud.vn/scheme : Bắt buộc phải có trong spec của tài nguyên Service : Khô","VKS cung cấp nhiều tính năng nổi bật như quản lý Control Plane tự động, hỗ trợ các phiên bản Kubernetes mới nhất, tích hợp Calico CNI, nâng cấp dễ dàng, tự động mở rộng và sửa lỗi, giảm chi phí và nân",Dịch vụ VKS (VNGCloud Kubernetes Service) có những tính năng nổi bật sau: Quản lý Control Plane hoàn toàn tự động (Fully Managed control plane): VKS giúp bạn giải phóng khỏi gánh nặng quản l,1.000000,0.785714,1.000000,0.917253,gemini-1.5-flash,InMemory
1,VKS là gì?,"1. Giới thiệu về VNGCloud Kubernetes Service - VKS 1.1. VNGCloud Kubernetes Service (VKS) là gì? VNG 3. Mô hình hoạt động của VKS VNGCloud Kubernetes Service (VKS) hỗ trợ 2 loại cluster chính là Public cho các tài nguyên Service có thuộc tính serviceType là LoadBalancer . Mặc định, một cụm Kubern VNGCloud Portal. vks.vngcloud.vn/scheme : Bắt buộc phải có trong spec của tài nguyên Service : Khô","VKS là viết tắt của VNGCloud Kubernetes Service, một dịch vụ quản lý Kubernetes được cung cấp bởi VNGCloud. VKS giúp bạn triển khai và quản lý các ứng dụng dựa trên container một cách dễ dàng và hiệu",VKS là viết tắt của VNGCloud Kubernetes Service. Nó là một dịch vụ quản lý Kubernetes được cung cấp bởi VNGCloud. VKS giúp bạn triển khai và quản lý các ứng dụng dựa trên container một,1.000000,0.800000,1.000000,0.945761,gemini-1.5-flash,InMemory
2,So sánh public và private cluster trong VKS,"3. Mô hình hoạt động của VKS VNGCloud Kubernetes Service (VKS) hỗ trợ 2 loại cluster chính là Public dụ: vks.vngcloud.vn/security-groups: secg-303d9c91-f4b2-400d-8e81-12039167da0c . Ngoài ra, bạn cũng VNGCloud Portal. vks.vngcloud.vn/scheme : Bắt buộc phải có trong spec của tài nguyên Service : Khô Security Group cho NLB của bạn bằng cách sử dụng dấu phẩy , để ngăn cách giữa các Security Group.","Public Cluster sử dụng địa chỉ IP Public để giao tiếp, trong khi Private Cluster sử dụng địa chỉ IP Private, dẫn đến Private Cluster có mức độ bảo mật cao hơn.","Public Cluster và Private Cluster là hai loại cụm Kubernetes chính được hỗ trợ bởi VKS. Mỗi loại có ưu nhược điểm riêng, phù hợp với những nhu cầu bảo mật và sử dụng khác nhau. Dưới đây là b",1.000000,0.750000,1.000000,0.733616,gemini-1.5-flash,InMemory
3,VKS có những tính năng gì nổi bật?,"1. Giới thiệu về VNGCloud Kubernetes Service - VKS 1.1. VNGCloud Kubernetes Service (VKS) là gì? VNG 3. Mô hình hoạt động của VKS VNGCloud Kubernetes Service (VKS) hỗ trợ 2 loại cluster chính là Public cho các tài nguyên Service có thuộc tính serviceType là LoadBalancer . Mặc định, một cụm Kubern VNGCloud Portal. vks.vngcloud.vn/scheme : Bắt buộc phải có trong spec của tài nguyên Service : Khô","VKS có các tính năng nổi bật như quản lý Control Plane tự động, hỗ trợ phiên bản Kubernetes mới nhất, tích hợp Calico CNI, nâng cấp dễ dàng, tự động mở rộng và sửa lỗi, giảm chi phí, tích hợp Blocksto",Dịch vụ VKS (VNGCloud Kubernetes Service) có những tính năng nổi bật sau: Quản lý Control Plane hoàn toàn tự động (Fully Managed control plane): VKS giúp bạn giải phóng khỏi gánh nặng quản l,1.000000,0.785714,1.000000,0.928116,deepseek-r1-distill-llama-70b,InMemory
4,VKS là gì?,"1. Giới thiệu về VNGCloud Kubernetes Service - VKS 1.1. VNGCloud Kubernetes Service (VKS) là gì? VNG 3. Mô hình hoạt động của VKS VNGCloud Kubernetes Service (VKS) hỗ trợ 2 loại cluster chính là Public cho các tài nguyên Service có thuộc tính serviceType là LoadBalancer . Mặc định, một cụm Kubern VNGCloud Portal. vks.vngclo